In [1]:
import pyodbc
import pandas as pd

conn = pyodbc.connect(
    driver='FreeTDS',
    server='lga-db4.pulse.data',
    port=58321,
    uid='sqoopuser',
    pwd='sqoop@207*',
    tds_version='7.3'
)

query = """
SELECT * FROM Utility.dbo.ComparePage;
"""

df = pd.read_sql_query(query, conn)

In [3]:
df[:10]

,Dt,TableName,PaidImpressions,LastModifiedDt
0,2017-09-09,lga-hive-factlogevent,151447969,2017-10-09 17:21:00
1,2017-09-09,lga-hive-logevent,151447969,2017-10-09 17:21:00
2,2017-09-09,lga-sql-advDailyStats,151447969,2017-10-09 17:21:00
3,2017-09-09,lga-sql-KPIHourly,151447969,2017-10-09 17:21:00
4,2017-09-09,lga-sql-pubDailyStats,151447969,2017-10-09 17:21:00
5,2017-09-09,sjc-hive-factlogevent,88819475,2017-10-09 17:21:00
6,2017-09-09,sjc-hive-logevent,151447969,2017-10-09 17:21:00
7,2017-09-09,sjc-sql-advDailyStats,151447969,2017-10-09 17:21:00
8,2017-09-09,sjc-sql-KPIHourly,151447969,2017-10-09 17:21:00
9,2017-09-09,sjc-sql-pubDailyStats,151447969,2017-10-09 17:21:00


In [8]:
list(df.TableName.unique())     # len is 12

[u'lga-hive-factlogevent',
 u'lga-hive-logevent',
 u'lga-sql-advDailyStats',
 u'lga-sql-KPIHourly',
 u'lga-sql-pubDailyStats',
 u'sjc-hive-factlogevent',
 u'sjc-hive-logevent',
 u'sjc-sql-advDailyStats',
 u'sjc-sql-KPIHourly',
 u'sjc-sql-pubDailyStats',
 u'vertica-kpihourly',
 u'vertica-rpt']

In [9]:
pivoted = pd.pivot_table(df, values='PaidImpressions', index=['Dt'], columns=['TableName'])

In [12]:
smalldf = pivoted.head()
smalldf

TableName,lga-hive-factlogevent,lga-hive-logevent,lga-sql-KPIHourly,lga-sql-advDailyStats,lga-sql-pubDailyStats,sjc-hive-factlogevent,sjc-hive-logevent,sjc-sql-KPIHourly,sjc-sql-advDailyStats,sjc-sql-pubDailyStats,vertica-kpihourly,vertica-rpt
Dt,,,,,,,,,,,,
2017-09-09,151447969.0,151447969.0,151447969.0,151447969.0,151447969.0,88819475.0,151447969.0,151447969.0,151447969.0,151447969.0,151447969.0,151447969.0
2017-09-10,162699891.0,162699891.0,162699891.0,162699891.0,162699891.0,79150905.0,162699891.0,162699891.0,162699891.0,162699891.0,162699891.0,162699891.0
2017-09-11,188552112.0,188552112.0,188552112.0,188552112.0,188552112.0,143116084.0,188552112.0,188552112.0,188552112.0,188552112.0,188552112.0,188552112.0
2017-09-12,167770656.0,167745381.0,167745381.0,167745381.0,167745381.0,111914535.0,167745381.0,167745381.0,167745381.0,167745381.0,167745381.0,167745381.0
2017-09-13,177078096.0,177078096.0,177078096.0,177078096.0,177078096.0,107435605.0,177078096.0,177078096.0,177078096.0,177078096.0,177078096.0,177078096.0


In [13]:
smalldf.shape

(5, 12)

In [17]:
smalldf.to_csv(sep='\t')

'Dt\tlga-hive-factlogevent\tlga-hive-logevent\tlga-sql-KPIHourly\tlga-sql-advDailyStats\tlga-sql-pubDailyStats\tsjc-hive-factlogevent\tsjc-hive-logevent\tsjc-sql-KPIHourly\tsjc-sql-advDailyStats\tsjc-sql-pubDailyStats\tvertica-kpihourly\tvertica-rpt\n2017-09-09\t151447969.0\t151447969.0\t151447969.0\t151447969.0\t151447969.0\t88819475.0\t151447969.0\t151447969.0\t151447969.0\t151447969.0\t151447969.0\t151447969.0\n2017-09-10\t162699891.0\t162699891.0\t162699891.0\t162699891.0\t162699891.0\t79150905.0\t162699891.0\t162699891.0\t162699891.0\t162699891.0\t162699891.0\t162699891.0\n2017-09-11\t188552112.0\t188552112.0\t188552112.0\t188552112.0\t188552112.0\t143116084.0\t188552112.0\t188552112.0\t188552112.0\t188552112.0\t188552112.0\t188552112.0\n2017-09-12\t167770656.0\t167745381.0\t167745381.0\t167745381.0\t167745381.0\t111914535.0\t167745381.0\t167745381.0\t167745381.0\t167745381.0\t167745381.0\t167745381.0\n2017-09-13\t177078096.0\t177078096.0\t177078096.0\t177078096.0\t177078096.0\t10